In [50]:
import polars as pl
import re

from pathlib import Path

In [2]:
DATA_PATH = Path('..', 'data')
list(DATA_PATH.glob('*'))

[WindowsPath('../data/input_1.txt')]

In [3]:
df = (
    pl
    .read_csv(
        DATA_PATH / 'input_1.txt',
        has_header=False,
    )
)

In [11]:
(
    df
    .with_columns(
        pl.col('column_1').str.extract_all('\d')
    )
    .select(
        pl
        .format(
            '{}{}',
            pl.col('column_1').list[0], pl.col('column_1').list[-1]
        )
        .cast(pl.Int32)
    )
    .sum()
)

column_1
i32
53651


In [69]:
# TODO: avoid .map_elements. How can I extract overlapping matches?

In [68]:
MAP = {
    'one': '1',
    'two': '2',
    'three': '3',
    'four': '4',
    'five': '5',
    'six': '6',
    'seven': '7',
    'eight': '8',
    'nine': '9',
}

(
    df
    .with_columns(
        pl
        .col('column_1')
        .map_elements(
            lambda x: re.findall(f"(?=(\d|{'|'.join(MAP.keys())}))", x)
        )
    )
    .with_columns(
        pl
        .format(
            '{}{}',
            pl.col('column_1').list[0].replace(MAP),
            pl.col('column_1').list[-1].replace(MAP),
        )
        .cast(pl.Int32)
    )
    .sum()
)

column_1
i32
53894
